In [ ]:
import pandas as pd
import os

# === Paths ===
# Input: The microbiome data we just created (Level 6)
microbiome_path = "/home/pintokf/Projects/Microbium/Mouses/MIPMLP_scripts/processed_subpca_level7.csv"

# Input: The metadata file
metadata_path = "/home/pintokf/Projects/Microbium/Mouses/mouses_2_data/metadata.txt"

# Output: The merged file ready for the next preprocessing steps
#output_path = "/home/pintokf/Projects/Microbium/Mouses/Ratio_model/merged_data_level6.csv"

print("--- Starting Merge Process ---")

# 1. Load Microbiome Data
try:
    df_micro = pd.read_csv(microbiome_path)
    print(f"Loaded Microbiome Data. Shape: {df_micro.shape}")
    # Verify ID exists
    if 'ID' not in df_micro.columns:
        raise ValueError("Column 'ID' not found in microbiome data!")
except Exception as e:
    print(f"Error loading microbiome data: {e}")
    exit(1)

# 2. Load Metadata
try:
    # Assuming metadata is tab-separated based on previous format
    df_meta = pd.read_csv(metadata_path, sep='\t')
    print(f"Loaded Metadata. Shape: {df_meta.shape}")
    
    # Standardize ID column name in Metadata
    # Common QIIME formats use '#SampleID' or 'SampleID'
    if '#SampleID' in df_meta.columns:
        df_meta.rename(columns={'#SampleID': 'ID'}, inplace=True)
    elif 'SampleID' in df_meta.columns:
        df_meta.rename(columns={'SampleID': 'ID'}, inplace=True)
        
    print(f"Metadata columns: {list(df_meta.columns)}")
    
except Exception as e:
    print(f"Error loading metadata: {e}")
    exit(1)

# 3. Merge Tables
# We use 'inner' merge to keep only samples that have BOTH microbiome and metadata info
merged_df = pd.merge(df_micro, df_meta, on='ID', how='inner')

print(f"--- Merge Complete ---")
print(f"Original Microbiome Samples: {len(df_micro)}")
print(f"Original Metadata Samples: {len(df_meta)}")
print(f"Merged Samples (Intersection): {len(merged_df)}")

# 4. Save
#merged_df.to_csv(output_path, index=False)
#print(f"✅ Saved merged file to: {output_path}")

--- Starting Merge Process ---
Loaded Microbiome Data. Shape: (72, 36)
Loaded Metadata. Shape: (72, 11)
Metadata columns: ['ID', 'barcode', '             LinkerPrimerSequence', 'ReversePrimer', 'Cage', 'MiceName', 'SamplingDate', 'AgeMonths', 'Death', 'DeathDate', 'DeathAgeMonths']
--- Merge Complete ---
Original Microbiome Samples: 72
Original Metadata Samples: 72
Merged Samples (Intersection): 72


In [ ]:
output_path = "/home/pintokf/Projects/Microbium/Mouses/Ratio_model_microbium/Preprocces_for_ratio_model/merged_data_level7.csv"
merged_df.to_csv(output_path, index=False)
print(f"✅ Saved merged file to: {output_path}")

✅ Saved merged file to: /home/pintokf/Projects/Microbium/Mouses/Ratio_model/Preprocces_for_ratio_model/merged_data_level6.csv


In [ ]:
# === Split Data based on 'Death' column ===

# 1. Create the Uncensored group (Death = yes)
df_uncensored = merged_df[merged_df['Death'] == 'yes'].copy()

# 2. Create the Censored group (Death = no)
df_censored = merged_df[merged_df['Death'] == 'no'].copy()

# === Verification ===
print(f"Original Total: {len(merged_df)}")
print(f"Uncensored (Dead/Events): {len(df_uncensored)}")
print(f"Censored (Alive/No Event): {len(df_censored)}")

# === Save to files ===
output_dir = "/home/pintokf/Projects/Microbium/Mouses/Ratio_model_microbium/Preprocces_for_ratio_model"

path_uncensored = f"{output_dir}/data_level7_uncensored.csv"
path_censored = f"{output_dir}/data_level7_censored.csv"

df_uncensored.to_csv(path_uncensored, index=False)
df_censored.to_csv(path_censored, index=False)

print(f"\n✅ Files saved successfully:")
print(f"1. {path_uncensored}")
print(f"2. {path_censored}")

Original Total: 72
Uncensored (Dead/Events): 22
Censored (Alive/No Event): 50

✅ Files saved successfully:
1. /home/pintokf/Projects/Microbium/Mouses/Ratio_model/Preprocces_for_ratio_model/data_level6_uncensored.csv
2. /home/pintokf/Projects/Microbium/Mouses/Ratio_model/Preprocces_for_ratio_model/data_level6_censored.csv


In [11]:
import pandas as pd

def process_mouse_data(df_input):
    df = df_input.copy()
    
    # === CRITICAL FIX: Clean column names ===
    # This removes the huge whitespace from '             LinkerPrimerSequence'
    df.columns = df.columns.str.strip()
    
    # 1. Drop unnecessary columns
    # Now that the columns are stripped, 'LinkerPrimerSequence' will match exactly
    cols_to_drop = ['barcode', 'LinkerPrimerSequence', 'ReversePrimer', 'Death']
    df = df.drop(columns=cols_to_drop, errors='ignore')
    
    # 2. Fix AgeMonths (remove '_months', convert to number)
    if 'AgeMonths' in df.columns:
        df['AgeMonths'] = df['AgeMonths'].astype(str).str.replace('_months', '', regex=False)
        df['AgeMonths'] = pd.to_numeric(df['AgeMonths'], errors='coerce')
    
    # 3. Fix SamplingDate
    # Logic: "20-May" -> Day: 01, Month: May, Year: 2020
    if 'SamplingDate' in df.columns:
        try:
            def fix_date_format(val):
                val_str = str(val).strip()
                if '-' in val_str:
                    parts = val_str.split('-')
                    # parts[0] = "20" (Year)
                    # parts[1] = "May" (Month)
                    year_suffix = parts[0]
                    month_name = parts[1]
                    # Construct string: "01-May-2020"
                    return f"01-{month_name}-20{year_suffix}"
                return val # Return original if format is unexpected

            # Apply the string formatting
            df['SamplingDate'] = df['SamplingDate'].apply(fix_date_format)
            
            # Convert to actual datetime object
            df['SamplingDate'] = pd.to_datetime(df['SamplingDate'], format='%d-%b-%Y')
            
        except Exception as e:
            print(f"⚠️ Warning: Date conversion failed: {e}")

    return df

# === Execution ===

print("--- Processing Uncensored (Dead) Data ---")
# Ensure df_uncensored exists from previous cells
df_uncensored_clean = process_mouse_data(df_uncensored)

# Debug prints to verify
print("Columns remaining:", list(df_uncensored_clean.columns))
print("Date Preview:")
print(df_uncensored_clean[['ID', 'SamplingDate']].head())


print("\n--- Processing Censored (Alive) Data ---")
# Ensure df_censored exists from previous cells
df_censored_clean = process_mouse_data(df_censored)
#df_censored_clean = df_censored_clean.drop('Cage', axis=1)
print("Columns remaining:", list(df_censored_clean.columns))


# === Save Files ===
df_uncensored_clean.to_csv(path_uncensored, index=False)
df_censored_clean.to_csv(path_censored, index=False)

print(f"\n✅ Files saved successfully.")

--- Processing Uncensored (Dead) Data ---
Columns remaining: ['ID', 'k__Bacteria_0;p__;c__;o__;f__;g__;s__', 'k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales_0;f__;g__;s__', 'k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides_0;s__', 'k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Porphyromonadaceae;g__Parabacteroides_0;s__', 'k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Prevotellaceae;g__Prevotella_0;s__', 'k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Rikenellaceae_0;g__;s__', 'k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Rikenellaceae;g__Alistipes_0;s__', 'k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__S24-7_0;g__;s__', 'k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__[Odoribacteraceae];g__Odoribacter_0;s__', 'k__Bacteria;p__Cyanobacteria;c__4C0d-2;o__YS2_0;f__;g__;s__', 'k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f

In [12]:
import pandas as pd

# === Logic to fix DeathDate specific format ===
# Input examples: "05_21_b", "06_21"
# Output: 01-05-2021 (datetime object)
def fix_death_date_format(val):
    val_str = str(val).strip()
    
    if val_str == 'nan' or val_str == '':
        return pd.NaT
    
    try:
        # Split by underscore '_'
        parts = val_str.split('_')
        
        # We need at least the first two parts (Month and Year)
        if len(parts) >= 2:
            month = parts[0]      # e.g., "05"
            year_short = parts[1] # e.g., "21"
            
            # Construct the string "01-MM-20YY"
            date_str = f"01-{month}-20{year_short}"
            
            # Convert to datetime
            return pd.to_datetime(date_str, format='%d-%m-%Y')
        else:
            return pd.NaT
            
    except Exception as e:
        print(f"⚠️ Error parsing date: {val} -> {e}")
        return pd.NaT

# === Main Processing for Uncensored Data ===
print("--- Processing Uncensored Data (Dates & Diff) ---")

# Ensure the dataframe exists
if 'df_uncensored_clean' in locals():
    
    # 1. Apply DeathDate Fix
    if 'DeathDate' in df_uncensored_clean.columns:
        df_uncensored_clean['DeathDate'] = df_uncensored_clean['DeathDate'].apply(fix_death_date_format)

    # 2. Calculate 'diff' Column
    # Formula: (DeathAgeMonths - AgeMonths) * 30
    if 'DeathAgeMonths' in df_uncensored_clean.columns and 'AgeMonths' in df_uncensored_clean.columns:
        
        # Ensure columns are numeric
        df_uncensored_clean['DeathAgeMonths'] = pd.to_numeric(df_uncensored_clean['DeathAgeMonths'], errors='coerce')
        df_uncensored_clean['AgeMonths'] = pd.to_numeric(df_uncensored_clean['AgeMonths'], errors='coerce')
        
        # Perform calculation
        df_uncensored_clean['diff'] = (df_uncensored_clean['DeathAgeMonths'] - df_uncensored_clean['AgeMonths']) * 30
        
        print("✅ Added 'diff' column.")
        # Preview the new column
        print(df_uncensored_clean[['ID', 'AgeMonths', 'DeathAgeMonths', 'diff']].head())
    else:
        print("❌ Error: Missing 'DeathAgeMonths' or 'AgeMonths' columns.")

    # 3. Save to file
    df_uncensored_clean = df_uncensored_clean.drop('DeathAgeMonths', axis=1)
    df_uncensored_clean.to_csv(path_uncensored, index=False)
    print(f"✅ Saved updated file to: {path_uncensored}")

else:
    print("❌ Error: df_uncensored_clean is not defined. Please run previous steps.")

--- Processing Uncensored Data (Dates & Diff) ---
✅ Added 'diff' column.
           ID  AgeMonths  DeathAgeMonths  diff
1   14-1_5-20          4              17   390
7   20-1_5-20          4              12   240
8   21-1_5-20          4              15   330
15  24-0_5-20          4              17   390
17  24-2_5-20          4              17   390
✅ Saved updated file to: /home/pintokf/Projects/Microbium/Mouses/Ratio_model/Preprocces_for_ratio_model/data_level6_uncensored.csv


In [13]:
import pandas as pd
from pandas.tseries.offsets import DateOffset

print("--- Processing Censored Data (Columns & Dates) ---")

# Ensure the dataframe exists
if 'df_censored_clean' in locals():
    
    # 1. Drop irrelevant columns for censored data
    cols_to_drop = ['DeathDate', 'DeathAgeMonths']
    df_censored_clean = df_censored_clean.drop(columns=cols_to_drop, errors='ignore')
    
    # Ensure numeric types for calculation
    if 'AgeMonths' in df_censored_clean.columns:
        df_censored_clean['AgeMonths'] = pd.to_numeric(df_censored_clean['AgeMonths'], errors='coerce')
        
        # 2. Calculate 'diff' column
        # Formula: (18 - AgeMonths) * 30
        df_censored_clean['diff'] = (18 - df_censored_clean['AgeMonths']) * 30
        
        # 3. Calculate 'DateEnd' column
        # Logic: SamplingDate + (18 - AgeMonths) months
        if 'SamplingDate' in df_censored_clean.columns:
            # Ensure SamplingDate is datetime
            df_censored_clean['SamplingDate'] = pd.to_datetime(df_censored_clean['SamplingDate'])
            
            # Define a helper function to add months per row
            def add_months_to_reach_18(row):
                try:
                    months_to_add = int(18 - row['AgeMonths'])
                    return row['SamplingDate'] + DateOffset(months=months_to_add)
                except Exception as e:
                    return pd.NaT

            # Apply calculation
            df_censored_clean['DateEnd'] = df_censored_clean.apply(add_months_to_reach_18, axis=1)
            
            # Ensure final format is datetime
            df_censored_clean['DateEnd'] = pd.to_datetime(df_censored_clean['DateEnd'])
            
            print("✅ Added 'diff' and 'DateEnd' columns.")
            print(df_censored_clean[['ID', 'AgeMonths', 'SamplingDate', 'DateEnd', 'diff']].head())
            
        else:
            print("❌ Error: 'SamplingDate' column missing.")
    else:
        print("❌ Error: 'AgeMonths' column missing.")

    # 4. Save to file
    df_censored_clean.to_csv(path_censored, index=False)
    print(f"✅ Saved updated censored file to: {path_censored}")

else:
    print("❌ Error: df_censored_clean is not defined.")

--- Processing Censored Data (Columns & Dates) ---
✅ Added 'diff' and 'DateEnd' columns.
          ID  AgeMonths SamplingDate    DateEnd  diff
0  14-0_5-20          4   2020-05-01 2021-07-01   420
2  14-2_5-20          4   2020-05-01 2021-07-01   420
3  15-1_5-20          4   2020-05-01 2021-07-01   420
4  18-1_5-20          4   2020-05-01 2021-07-01   420
5  18-2_5-20          4   2020-05-01 2021-07-01   420
✅ Saved updated censored file to: /home/pintokf/Projects/Microbium/Mouses/Ratio_model/Preprocces_for_ratio_model/data_level6_censored.csv


In [14]:
df_uncensored_clean.rename(columns={'SamplingDate': 'Date'}, inplace=True)
df_censored_clean.rename(columns={'SamplingDate': 'Date'}, inplace=True)
df_uncensored_clean.to_csv(path_uncensored, index=False)
df_censored_clean.to_csv(path_censored, index=False)